# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm

In [ ]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [ ]:
config = pd.read_csv('config.csv').iloc[0]

In [ ]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [ ]:
user = config['id']
k_value = config['k']

### Read the Data

In [ ]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [ ]:
user_likes = dict()
different_image=dict()
dict_to_store_index=dict()
index=0
index_key=0
key_user=-1
for line in appreciate_data:
  line = line.strip()
  user_id = int(line.split()[0])
  item_id = int(line.split()[1])
  if user_id not in user_likes:
    user_likes[user_id] = list()
    if user_id==user:
      key_user=index_key
    dict_to_store_index[index_key]=user_id
    index_key=index_key+1
  user_likes[user_id].append(item_id)
  if item_id not in different_image:
    different_image[item_id]=index
    index=index+1
  #user_likes[user_id].append(item_id)
#user_likes
#user_likes
#print(len(different_image))
#len(user_likes)
#print(user)
#len(user_likes)


In [ ]:
diff_user=len(user_likes)
diff_image=len(different_image)
dense_matrix=np.zeros((diff_user,diff_image),dtype=np.int8)



In [ ]:
row=0
for i in list(user_likes.keys()):
  for j in user_likes[i]:
    dense_matrix[row][different_image[j]]=1
  row=row+1

In [ ]:
from scipy.sparse import csr_matrix
sparse_mat=csr_matrix(dense_matrix)
del(dense_matrix)

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)
sparse_matrix=svd.fit_transform(sparse_mat)

### Use KNN after Collaborative Filtering to find nearest neighbors

In [ ]:
def neighbors(user,k_value):  
  i=0
  dict_of_val=dict()
  for row in sparse_matrix:
    this_user=dict_to_store_index[i]
    if this_user!=user:
      result= dot(sparse_matrix[i], sparse_matrix[key_user])/(norm(sparse_matrix[i])*norm(sparse_matrix[key_user]))
      #result=1-spatial.distance.cosine(sparse_matrix[i],sparse_matrix[key_user])  #passing ndarray
      if result not in dict_of_val:
        dict_of_val[result]=list()
      dict_of_val[result].append(i)
    i=i+1

  list_of_result=list(dict_of_val.keys())
  list_of_result.sort(reverse=True)

  main_list=list()
  k=k_value
  while k>0:
    for i in list_of_result:
      temp=list(dict_of_val[i])
      if k==0:
        break
      for j in temp:
        main_list.append(dict_to_store_index[j])
        k=k-1
        if k==0:
          break
  return main_list       

### Open the output file to write all the lines to the file

In [ ]:
outFile = open(config['output_file'], 'w')
for user_id in neighbors(user,k_value):
  #user_id = list(user_likes.keys())[n_user]
  main_set=set(user_likes[user])
  ultimate_set=set(user_likes[user_id])
  final_set=ultimate_set.difference(main_set)
  for item_id in final_set:
    outFile.write(str(item_id) + ' ' + str(user_id) + '\n')
outFile.close()